## feedforward spiking neural networks

In [1]:
import snntorch as snn
from snntorch import spikeplot as splt
from snntorch import spikegen

import torch
import torch.nn as nn
import matplotlib.pyplot as plt

In [ ]:
#@title Plotting Settings
def plot_cur_mem_spk(cur, mem, spk, thr_line=False, vline=False, title=False, ylim_max1=1.25, ylim_max2=1.25):
  # Generate Plots
  fig, ax = plt.subplots(3, figsize=(8,6), sharex=True,
                        gridspec_kw = {'height_ratios': [1, 1, 0.4]})

  # Plot input current
  ax[0].plot(cur, c="tab:orange")
  ax[0].set_ylim([0, ylim_max1])
  ax[0].set_xlim([0, 200])
  ax[0].set_ylabel("Input Current ($I_{in}$)")
  if title:
    ax[0].set_title(title)

  # Plot membrane potential
  ax[1].plot(mem)
  ax[1].set_ylim([0, ylim_max2])
  ax[1].set_ylabel("Membrane Potential ($U_{mem}$)")
  if thr_line:
    ax[1].axhline(y=thr_line, alpha=0.25, linestyle="dashed", c="black", linewidth=2)
  plt.xlabel("Time step")

  # Plot output spike using spikeplot
  splt.raster(spk, ax[2], s=400, c="black", marker="|")
  if vline:
    ax[2].axvline(x=vline, ymin=0, ymax=6.75, alpha = 0.15, linestyle="dashed", c="black", linewidth=2, zorder=0, clip_on=False)
  plt.ylabel("Output spikes")
  plt.yticks([])

  plt.show()

def plot_snn_spikes(spk_in, spk1_rec, spk2_rec, title):
  # Generate Plots
  fig, ax = plt.subplots(3, figsize=(8,7), sharex=True,
                        gridspec_kw = {'height_ratios': [1, 1, 0.4]})

  # Plot input spikes
  splt.raster(spk_in[:,0], ax[0], s=0.03, c="black")
  ax[0].set_ylabel("Input Spikes")
  ax[0].set_title(title)

  # Plot hidden layer spikes
  splt.raster(spk1_rec.reshape(num_steps, -1), ax[1], s = 0.05, c="black")
  ax[1].set_ylabel("Hidden Layer")

  # Plot output spikes
  splt.raster(spk2_rec.reshape(num_steps, -1), ax[2], c="black", marker="|")
  ax[2].set_ylabel("Output Spikes")
  ax[2].set_ylim([0, 10])

  plt.show()

In [ ]:
def leaky_integrate_and_fire(mem, x, w, beta, threshold=1):
  spk = (mem > threshold) # if membrane exceeds threshold, spk=1, else, 0
  mem = beta * mem + w*x - spk*threshold # U[t+1]=beta * U[t] + W*X[t+1]-S[t]*U_thr
  return spk, mem

# set neuronal parameters
delta_t = torch.tensor(1e-3)
tau = torch.tensor(5e-3)
beta = torch.exp(-delta_t/tau)

print(f"The decay rate is: {beta:.3f}")

In [ ]:
num_steps = 200

# initialize inputs/outputs + small step current input
x = torch.cat((torch.zeros(10), torch.ones(190)*0.5), 0)
mem = torch.zeros(1)
spk_out = torch.zeros(1)
mem_rec = []
spk_rec = []

# neuron parameters
w = 0.4
beta = 0.819

# neuron simulation
for step in range(num_steps):
  spk, mem = leaky_integrate_and_fire(mem, x[step], w=w, beta=beta)
  mem_rec.append(mem)
  spk_rec.append(spk)

# convert lists to tensors
mem_rec = torch.stack(mem_rec)
spk_rec = torch.stack(spk_rec)

plot_cur_mem_spk(x*w, mem_rec, spk_rec, thr_line=1,ylim_max1=0.5,
                 title="LIF Neuron Model With Weighted Step Voltage")

In [ ]:
lif1 = snn.Leaky(beta=0.8)

In [ ]:
w = 0.21
cur_in = torch.cat((torch.zeros(10), torch.ones(190)*w), 0)
mem = torch.zeros(1)
spk = torch.zeros(1)
mem_rec = []
spk_rec = []

for step in range(num_steps):
  spk, mem = lif1(cur_in[step], mem)
  mem_rec.append(mem)
  spk_rec.append(spk)

mem_rec = torch.stack(mem_rec)
spk_rec = torch.stack(spk_rec)

plot_cur_mem_spk(cur_in, mem_rec, spk_rec, thr_line=1, ylim_max1=0.5, title='snn.Leaky Neuron Model')

### 3-layer SNN  
pytorch : neuron간 연결  
snntorch : create neuron

In [ ]:
# layer
num_inputs = 784
num_hidden = 1000
num_outputs = 10
beta = 0.99

# initialize
fc1 = nn.Linear(num_inputs, num_hidden)
lif1 = snn.Leaky(beta=beta)
fc2 = nn.Linear(num_hidden, num_outputs)
lif2 = snn.Leaky(beta=beta)

In [ ]:
# hidden state
mem1 = lif1.init_leaky() #initialization
mem2 = lif2.init_leaky()

mem2_rec = []
spk1_rec = []
spk2_rec = []

In [ ]:
spk_in = spikegen.rate_conv(torch.rand((200, 784))).unsqueeze(1)
print(f'DImensions of spk_in: {spk_in.size()}') # time * batch_size * feature_dimension
# unsqueeze : 1차원으로 연결 -> bath_size=1이어야

### pytorch & snntorch
pytorch : neuron 연결  
snntorch : 결과를 spiking model로

$X_{ij}=0 or 1$: 입력스파이크. i번째 input, j번째 neuron  
$I_j = \sum_i (W_{ij} X_i)$
nn.Linear : W를 생성하여 $I_{in}$ scaling  

nn.Linear : 가중치가 무작위  
=> dead neuron 발생 가능 => 학습 불가
가중치 초기화 & 하이퍼파라미터 설정 잘해야돼

In [ ]:
for step in range(num_steps):
  cur1 = fc1(spk_in[step])
  spk1, mem1 = lif1(cur1, mem1)
  cur2 = fc2(spk1)
  spk2, mem2 = lif2(cur2, mem2)

  mem2_rec.append(mem2)
  spk1_rec.append(spk1)
  spk2_rec.append(spk2)

mem2_rec = torch.stack(mem2_rec)
spk1_rec = torch.stack(spk1_rec)
spk2_rec = torch.stack(spk2_rec)

plot_snn_spikes(spk_in, spk1_rec, spk2_rec, "Fully Connected Spiking Neural Networks")

In [ ]:
from IPython.display import HTML

fig, ax = plt.subplots(facecolor='w', figsize=(12,7))
labels=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
spk2_rec = spk2_rec.squeeze(1).detach().cpu()

anim = splt.spike_count(spk2_rec, fig, ax, labels=labels, animate=True)
HTML(anim.to_html5_video())

In [ ]:
splt.traces(mem2_rec.squeeze(1), spk=spk2_rec.squeeze(1)) # membrane potential trace
fig = plt.gcf()
fig.set_size_inches(8, 6)